In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("preprocessed_apartment_rentals_no_state_dummies_nocluster.csv")
print(data.columns)
# Features und Ziel
X = data[["square_feet", "state", "bathrooms",  "bathrooms", "cityname"]]
y = data["price"]

# Numerische Spalten
num_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Skalieren
#scaler = StandardScaler()
#X[num_cols] = scaler.fit_transform(X[num_cols])


Index(['bathrooms', 'bedrooms', 'has_photo', 'price', 'square_feet',
       'cityname', 'state', 'latitude', 'longitude', 'time', 'pool', 'gym',
       'parking', 'sauna', 'elevator', 'clubhouse', 'source_Andere source',
       'source_GoSection8', 'source_ListedBuy', 'source_RealRentals',
       'source_RentDigs.com', 'source_RentLingo'],
      dtype='object')


In [5]:
from Nested_CV_targetEncoding import NestedCVRegressorWithTargetEncoding
from sklearn.linear_model import Ridge
import numpy as np
ridge_params = {"alpha": np.logspace(-3, 3, 10)}

ridge_cv = NestedCVRegressorWithTargetEncoding(model = Ridge(), param_grid = ridge_params, encode_cols=["state"])
ridge_cv.run(X, y, output=True)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 1/5 | Best Params: {'model__alpha': np.float64(46.41588833612773)} | Fit Time: 0.128s | Outer R²: 0.505 | MSE: 234233.733
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 2/5 | Best Params: {'model__alpha': np.float64(0.001)} | Fit Time: 0.120s | Outer R²: 0.491 | MSE: 236210.745
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 3/5 | Best Params: {'model__alpha': np.float64(10.0)} | Fit Time: 0.110s | Outer R²: 0.498 | MSE: 233482.372
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 4/5 | Best Params: {'model__alpha': np.float64(0.001)} | Fit Time: 0.098s | Outer R²: 0.496 | MSE: 233979.194
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 5/5 | Best Params: {'model__alpha': np.float64(2.154434690031882)} | Fit Time: 0.100s | Outer R²: 0.499 | MSE: 232677.867


In [13]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from Nested_CV_targetEncoding import NestedCVRegressorWithTargetEncoding
import numpy as np

# Pipeline erstellen
poly_ridge = Pipeline([
    ("poly", PolynomialFeatures(degree=5, include_bias=False)),  # Grad 3
    ("scaler", StandardScaler()),                                  # optional, stabilisiert die Regression
    ("ridge", Ridge())
])

# Hyperparameter Grid für Ridge
ridge_params = {"ridge__alpha": np.logspace(-3, 3, 10)}  # 'ridge__' prefix wegen Pipeline

# Nested CV mit Target Encoding für Spalte "state"
ridge_poly_cv = NestedCVRegressorWithTargetEncoding(
    model=poly_ridge,
    param_grid=ridge_params,
    encode_cols=["state", "cityname"],  # Target Encoding für die Stadt
    outer_splits=5,
    inner_splits=5
)

# Run
ridge_poly_cv.run(X, y, output=True)

# Ergebnisse
print("Mean R²:", ridge_poly_cv.get_mean_r2())
print("Mean MSE:", ridge_poly_cv.get_mean_mse())
print("Best Params:", ridge_poly_cv.get_best_params())


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 1/5 | Best Params: {'model__ridge__alpha': np.float64(1000.0)} | Fit Time: 0.739s | Outer R²: 0.701 | MSE: 141453.977
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 2/5 | Best Params: {'model__ridge__alpha': np.float64(1000.0)} | Fit Time: 1.578s | Outer R²: 0.686 | MSE: 145817.946
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 3/5 | Best Params: {'model__ridge__alpha': np.float64(215.44346900318823)} | Fit Time: 1.136s | Outer R²: 0.690 | MSE: 144136.997
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 4/5 | Best Params: {'model__ridge__alpha': np.float64(215.44346900318823)} | Fit Time: 3.018s | Outer R²: 0.687 | MSE: 145205.341
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Outer Fold 5/5 | Best Params: {'model__ridge__alpha': np.float64(215.44346900318823)} | Fit Time: 4.798s | Outer R²: 0.694 | MSE: 142091.845
Mean R²: 0.691